<a href="https://colab.research.google.com/github/ZekeriyaYavan/Kolektif-Proje/blob/main/kolektif_proje_(7)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# GOOGLE DRIVE BAĞLAMA (ZORLA / FORCE)
# ==========================================
from google.colab import drive
import os

print("🔗 Google Drive bağlanıyor (Force Mode)...")

# force_remount=True ekledik, bu sayede "içi dolu" hatasını ezip geçer.
drive.mount('/content/drive', force_remount=True)

# Proje ana klasörü
PROJECT_DIR = "/content/drive/MyDrive/Proje_Distill"
os.makedirs(PROJECT_DIR, exist_ok=True)

print("📁 Proje klasörü:", PROJECT_DIR)
print("✅ Drive başarıyla bağlandı!")

🔗 Google Drive bağlanıyor (Force Mode)...
Mounted at /content/drive
📁 Proje klasörü: /content/drive/MyDrive/Proje_Distill
✅ Drive başarıyla bağlandı!


In [ ]:
# ==========================================
# TORCH FIX (libtorch_global_deps.so hatası)
# ==========================================
!pip uninstall -y -q torch torchvision torchaudio

# Colab GPU için stabil torch kurulumu
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

print("✅ Torch yeniden kuruldu. Şimdi Runtime Restart yapmalısın.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 131.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 604.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch

print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())


torch: 2.5.1+cu121
cuda available: True


In [ ]:
import os

# Proje klasörü (Drive içinde)
BASE_PATH = "/content/drive/MyDrive/Proje_Distill"

# Dosyalar
INPUT_FILE = os.path.join(BASE_PATH, "train_2000.jsonl")
TEST_FILE  = os.path.join(BASE_PATH, "test_500.jsonl")

print("📁 BASE_PATH:", BASE_PATH)
print("📄 INPUT_FILE:", INPUT_FILE)

# Klasör yoksa oluştur
os.makedirs(BASE_PATH, exist_ok=True)

# Dosya var mı kontrol
print("✅ train_2000.jsonl var mı?", os.path.exists(INPUT_FILE))
print("✅ test_500.jsonl var mı?", os.path.exists(TEST_FILE))


📁 BASE_PATH: /content/drive/MyDrive/Proje_Distill
📄 INPUT_FILE: /content/drive/MyDrive/Proje_Distill/train_2000.jsonl
✅ train_2000.jsonl var mı? True
✅ test_500.jsonl var mı? True


In [ ]:
# ==========================================
# HÜCRE 2: VERİ HAZIRLIĞI (GSM8K - TÜRKÇE)
# ==========================================
import json
import random
import os
from datasets import load_dataset

# Eğer önceki hücrede PROJECT_DIR tanımlıysa bunu kullan:
BASE_PATH = "/content/drive/MyDrive/Proje_Distill"
os.makedirs(BASE_PATH, exist_ok=True)

# Sabit Seed (Her denemede aynı soruların gelmesi için)
SEED = 42
random.seed(SEED)

print("📥 GSM8K TR (ytu-ce-cosmos) veri seti indiriliyor...")

# 1) Türkçe Veri Setini Yükle
try:
    dataset = load_dataset("ytu-ce-cosmos/gsm8k_tr", split="train")
    print(f"✅ Veri indirildi. Toplam Soru Sayısı: {len(dataset)}")

    # Veri setinin formatını kontrol edelim
    print(f"🔍 İlk veri örneği: {dataset[0]}")

    all_data = list(dataset)

except Exception as e:
    print(f"❌ Veri seti hatası: {e}")
    all_data = []

if all_data:
    # 2) Verileri Karıştır
    indices = list(range(len(all_data)))
    random.shuffle(indices)

    # 3) Eğitim (2000) ve Test (500) Olarak Ayır
    TRAIN_SIZE = 2000
    TEST_SIZE = 500

    train_data = [all_data[i] for i in indices[:TRAIN_SIZE]]
    test_data  = [all_data[i] for i in indices[TRAIN_SIZE:TRAIN_SIZE + TEST_SIZE]]

    # 4) Kaydetme Fonksiyonu
    def save_jsonl(data, filename):
        path = os.path.join(BASE_PATH, filename)
        written = 0

        with open(path, "w", encoding="utf-8") as f:
            for item in data:
                q_text = item.get("question")
                a_text = item.get("answer")

                if not q_text or not a_text:
                    continue

                record = {
                    "question": q_text,
                    "answer": a_text
                }

                f.write(json.dumps(record, ensure_ascii=False) + "\n")
                written += 1

        print(f"💾 Kaydedildi: {path} | satır: {written}")

    # Dosyaları Oluştur
    save_jsonl(train_data, "train_2000.jsonl")
    save_jsonl(test_data, "test_500.jsonl")

    print(f"\n✅ Veri Setleri Hazır! (Train: {len(train_data)}, Test: {len(test_data)})")
else:
    print("⚠️ Veri seti boş olduğu için işlem yapılamadı.")


📥 GSM8K TR (ytu-ce-cosmos) veri seti indiriliyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

gsm8k_tr.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/8792 [00:00<?, ? examples/s]

✅ Veri indirildi. Toplam Soru Sayısı: 8792
🔍 İlk veri örneği: {'question': 'Borris tekel bayisi her 6 ayda bir 90 kilogram üzüm kullanıyor. Üretimini yüzde yirmi oranında artırmayı düşünüyor. Üretimini artırdıktan sonra bir yıl içinde kaç üzüme ihtiyacı olur?', 'answer': 'Borris şu anda her 6 ayda 90 kilogram üzüm kullanıyorsa, bir yılda 180 kilogram üzüm kullanıyor demektir. Üretimini yüzde 20 oranında artırırsa:\n\n180 kilogram x 1.20 = 216 kilogram\n\nBuna göre, üretimini artırdıktan sonra bir yıl içinde 216 kilogram üzüme ihtiyacı olacaktır.'}
💾 Kaydedildi: /content/drive/MyDrive/Proje_Distill/train_2000.jsonl | satır: 2000
💾 Kaydedildi: /content/drive/MyDrive/Proje_Distill/test_500.jsonl | satır: 500

✅ Veri Setleri Hazır! (Train: 2000, Test: 500)


In [ ]:
# ==========================================
# 1. ADIM: KÜTÜPHANE GÜNCELLEME (ZORUNLU)
# ==========================================
print("🔄 Kütüphaneler güncelleniyor (GitHub Sürümü)...")

# Standart pip install yetmez, en son geliştirici sürümünü almamız lazım
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U accelerate bitsandbytes sentencepiece

print("✅ Güncelleme tamamlandı. ŞİMDİ RUNTIME RESTART YAPMALISIN!")

🔄 Kütüphaneler güncelleniyor (GitHub Sürümü)...
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-kkf80adv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-kkf80adv
  Resolved https://github.com/huggingface/transformers.git to commit 9413239102cb6eee5c131236ba4aeda403217572
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 18.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.1.dev0-py3-none-any.whl size=11231730 sha256=19881b281f07c2cdb5bf38d3ee002bad02df89ae2056c577d19c14aea68aaa16
  Stored in directory: /tmp/pip-ephem-wheel-cache-tqacca04/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggi

In [ ]:
!pip install --upgrade huggingface_hub transformers

In [ ]:
# ==========================================
# HÜCRE 1: SETUP & FONKSİYONLAR (TURBO BATCH)
# ==========================================
import torch, gc, json, os, re, time
from transformers import AutoTokenizer, AutoModelForCausalLM

BATCH_SIZE = 8
MAX_NEW_TOKENS = 450
RETRY_TOKENS = 64

BASE_PATH = "/content/drive/MyDrive/Proje_Distill"
INPUT_FILE = os.path.join(BASE_PATH, "train_2000.jsonl")
os.makedirs(BASE_PATH, exist_ok=True)

def extract_final_number(text: str):
    if not text: return None
    m = re.search(r"Final\s*Answer\s*:?\s*(.*)", text, flags=re.IGNORECASE)
    if m:
        tail = m.group(1).strip()
        m2 = re.search(r"([-+]?\d+(?:\.\d+)?)", tail)
        if m2: return m2.group(1)

    m = re.search(r"(Cevap|Sonuç)\s*:?\s*(.*)", text, flags=re.IGNORECASE)
    if m:
        tail = m.group(2).strip()
        m2 = re.search(r"([-+]?\d+(?:\.\d+)?)", tail)
        if m2: return m2.group(1)
    return None

def fallback_last_number(text: str):
    nums = re.findall(r"([-+]?\d+(?:\.\d+)?)", text)
    if not nums: return None
    return nums[-1]

def fix_format(answer: str):
    if answer is None: return None, None
    answer = answer.strip()

    final = extract_final_number(answer)
    if final is None:
        final = fallback_last_number(answer)
    if final is None:
        return None, None

    m2 = re.search(r"<think>(.*?)</think>", answer, flags=re.DOTALL | re.IGNORECASE)
    if m2:
        think = m2.group(1).strip()
    else:
        # think yoksa: tüm metni düşünce kabul et
        think = answer.strip()

    think = think.replace("<think>", "").replace("</think>", "").strip()
    think = re.sub(r"\n{3,}", "\n\n", think)

    formatted = f"<think>\n{think}\n</think>\nFinal Answer: {final}"
    return formatted, final

def build_chat_messages(question: str):
    example_output = (
        "<think>\n"
        "Alan = Uzunluk x Genişlik\n"
        "Alan = 10 * 5 = 50\n"
        "</think>\n"
        "Final Answer: 50"
    )

    user_msg = (
        f"Soru: {question}\n\n"
        "GÖREV: Adım adım düşün ve soruyu çöz.\n"
        "KURALLAR:\n"
        "1. Önce <think> etiketleri arasında işlemleri yap.\n"
        "2. Bulduğun sonucu en sona 'Final Answer: <sayı>' formatında yaz.\n"
        "3. ⚠️ DİKKAT: Aşağıdaki örnek sadece formatı göstermek içindir. Örnekteki sayıları ASLA kopyalama! Kendi bulduğun sonucu yaz!\n\n"
        "ÖRNEK FORMAT:\n"
        f"{example_output}"
    )
    return [{"role": "user", "content": user_msg}]

@torch.no_grad()
def run_teacher_specific(teacher_name, model_id):
    output_path = os.path.join(BASE_PATH, f"distill_{teacher_name}.jsonl")
    print(f"\n🚀 BAŞLIYOR (BATCH MODU): {teacher_name}")
    print(f"💾 Output: {output_path}")

    if not os.path.exists(INPUT_FILE):
        print("❌ INPUT yok:", INPUT_FILE)
        return

    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        all_questions = [json.loads(line)["question"] for line in f]

    start_index = 0
    if os.path.exists(output_path):
        with open(output_path, "r", encoding="utf-8") as f:
            start_index = sum(1 for _ in f)
        print(f"   ⚠️ Resume: {start_index}/{len(all_questions)}")

    if start_index >= len(all_questions):
        print("   ✅ Bu model için işlem tamamlanmış.")
        return

    print("   -> Model yükleniyor...")
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    tokenizer.padding_side = "left"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.float16
    )
    model.eval()

    t0 = time.time()
    saved, skipped = 0, 0

    with open(output_path, "a", encoding="utf-8") as f_out:
        for i in range(start_index, len(all_questions), BATCH_SIZE):
            batch_q = all_questions[i:i+BATCH_SIZE]

            batch_prompts = []
            for q in batch_q:
                msgs = build_chat_messages(q)
                if hasattr(tokenizer, "apply_chat_template"):
                    txt = tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
                else:
                    txt = msgs[0]["content"]
                batch_prompts.append(txt)

            inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True).to(model.device)

            try:
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=MAX_NEW_TOKENS,
                    do_sample=False,
                    pad_token_id=tokenizer.pad_token_id
                )
            except Exception as e:
                print(f"❌ Batch generate hatası: {e}")
                continue

            # 🔥 EN KRİTİK DÜZELTME:
            # prompt'u kesip sadece "yeni üretilen kısmı" alıyoruz
            prompt_len = inputs["input_ids"].shape[1]
            gen_only = outputs[:, prompt_len:]
            generated_texts = tokenizer.batch_decode(gen_only, skip_special_tokens=True)

            for j, raw_text in enumerate(generated_texts):
                q_curr = batch_q[j]
                response, final = fix_format(raw_text)

                if response is None:
                    # Retry tekil
                    retry_text = build_retry_prompt(q_curr)
                    inp2 = tokenizer(retry_text, return_tensors="pt").to(model.device)
                    out2 = model.generate(
                        **inp2,
                        max_new_tokens=RETRY_TOKENS,
                        do_sample=False,
                        pad_token_id=tokenizer.pad_token_id
                    )
                    raw2 = tokenizer.decode(out2[0][inp2.input_ids.shape[1]:], skip_special_tokens=True)
                    response, final = fix_format(raw2)

                    if response is None:
                        skipped += 1
                        continue

                record = {
                    "teacher": teacher_name,
                    "model_id": model_id,
                    "question": q_curr,
                    "answer": response
                }
                f_out.write(json.dumps(record, ensure_ascii=False) + "\n")
                f_out.flush()  # 🔥 garanti yaz

                saved += 1

            # log
            if (i + BATCH_SIZE) % 20 < BATCH_SIZE:
                elapsed = (time.time() - t0) / 60
                speed = saved / elapsed if elapsed > 0 else 0
                print(f"   ✅ {i+len(batch_q)}/{len(all_questions)} | saved={start_index+saved} | skipped={skipped} | Hız: {speed:.1f} soru/dk")

    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()
    print(f"🎉 {teacher_name} BİTTİ!")


Error importing huggingface_hub._snapshot_download: cannot import name 'DryRunError' from 'huggingface_hub.errors' (/usr/local/lib/python3.12/dist-packages/huggingface_hub/errors.py)


ImportError: cannot import name 'DryRunError' from 'huggingface_hub.errors' (/usr/local/lib/python3.12/dist-packages/huggingface_hub/errors.py)

In [ ]:
# ==========================================
# TEACHER 1: COSMOS T1
# ==========================================

run_teacher_specific(
    teacher_name="Cosmos_T1",
    model_id="ytu-ce-cosmos/Turkish-Gemma-9b-T1"
)

import gc, torch
gc.collect()
torch.cuda.empty_cache()
print("🧹 GPU temizlendi, sıradakine hazır.")


In [ ]:
# ==========================================
# TEACHER 2: QWEN TURKISH
# ==========================================

# 1. Modeli Çalıştır
run_teacher_specific(
    teacher_name="Qwen_TR",
    model_id="hosasmek/qwen2.5-7b-instruct-reasoning-tr"
)

# 2. Temizlik Yap (Bitişte)
import gc, torch
gc.collect()
torch.cuda.empty_cache()
print("🧹 Qwen bitti, GPU temizlendi. Sıradakine geçebilirsiniz.")

In [ ]:
# ==========================================
# HÜCRE 2: KESİN ÇÖZÜM (GenerationMixin Patch)
# ==========================================

from transformers.generation.utils import GenerationMixin
import gc, torch

def run_teacher_versioned(teacher_name, model_id, temp, top_p, max_new_tokens, do_sample=True):
    """
    Hücre 1'e dokunmadan, GenerationMixin üzerindeki generate fonksiyonunu
    geçici olarak değiştirir. Bu yöntem en garantisidir.
    """

    # 1. Asıl generate fonksiyonunun gerçek sahibi GenerationMixin'dir.
    original_generate = GenerationMixin.generate

    # 2. Kendi ayarlarımızı zorlayan fonksiyonu yazıyoruz
    def patched_generate(self, *args, **kwargs):
        # Parametreleri eziyoruz (Override)
        kwargs["do_sample"] = do_sample
        kwargs["temperature"] = temp
        kwargs["top_p"] = top_p
        kwargs["max_new_tokens"] = max_new_tokens

        # Orijinal fonksiyonu yeni ayarlarla çağırıyoruz
        return original_generate(self, *args, **kwargs)

    # 3. YAMALAMA: Artık tüm modeller bu fonksiyonu kullanacak
    GenerationMixin.generate = patched_generate

    print(f"\n🔧 {teacher_name} AKTİF (GenerationMixin Patching):")
    print(f"   do_sample={do_sample} | temp={temp} | top_p={top_p} | max_new_tokens={max_new_tokens}")

    try:
        # Hücre 1'deki fonksiyonu çalıştır
        # (Bu fonksiyon model.generate çağırdığında aslında patched_generate çalışacak)
        run_teacher_specific(teacher_name=teacher_name, model_id=model_id)
    except Exception as e:
        print(f"❌ Hata: {e}")
    finally:
        # 4. TEMİZLİK: Her şeyi eski haline getir (Çok önemli!)
        GenerationMixin.generate = original_generate
        print("🔧 generate fonksiyonu orijinal haline döndü.")
        gc.collect()
        torch.cuda.empty_cache()

# --- V2 İÇİN ÇALIŞTIR ---
run_teacher_versioned(
    teacher_name="Cosmos_T1_v2",
    model_id="ytu-ce-cosmos/Turkish-Gemma-9b-T1",
    temp=0.6,
    top_p=0.9,
    max_new_tokens=256,
    do_sample=True
)

In [ ]:
# ==========================================
# HÜCRE 2: KESİN ÇÖZÜM (GenerationMixin Patch & Eksik Fonksiyon)
# ==========================================

from transformers.generation.utils import GenerationMixin
import gc, torch
import sys  # Gerekirse modül seviyesinde eklemek için

# 1. Eksik Fonksiyonu Tanımlıyoruz
def build_retry_prompt(question):
    return f"Soru: {question}\n\nÖnceki cevap formatı hatalıydı. Lütfen kurallara tam uyarak tekrar cevapla."

# 2. Fonksiyonu Global Alana Ekliyoruz (Bu kritik!)
# Böylece Hücre 1'deki kod bu ismi aradığında bulabilecek.
globals()['build_retry_prompt'] = build_retry_prompt

def run_teacher_versioned(teacher_name, model_id, temp, top_p, max_new_tokens, do_sample=True):
    """
    Hücre 1'e dokunmadan, GenerationMixin üzerindeki generate fonksiyonunu
    geçici olarak değiştirir. Bu yöntem en garantisidir.
    """

    # Asıl generate fonksiyonunun gerçek sahibi GenerationMixin'dir.
    original_generate = GenerationMixin.generate

    # Kendi ayarlarımızı zorlayan fonksiyonu yazıyoruz
    def patched_generate(self, *args, **kwargs):
        # Parametreleri eziyoruz (Override)
        kwargs["do_sample"] = do_sample
        kwargs["temperature"] = temp
        kwargs["top_p"] = top_p
        kwargs["max_new_tokens"] = max_new_tokens

        # Orijinal fonksiyonu yeni ayarlarla çağırıyoruz
        return original_generate(self, *args, **kwargs)

    # YAMALAMA: Artık tüm modeller bu fonksiyonu kullanacak
    GenerationMixin.generate = patched_generate

    print(f"\n🔧 {teacher_name} AKTİF (GenerationMixin Patching):")
    print(f"   do_sample={do_sample} | temp={temp} | top_p={top_p} | max_new_tokens={max_new_tokens}")

    try:
        # Hücre 1'deki fonksiyonu çalıştır
        run_teacher_specific(teacher_name=teacher_name, model_id=model_id)
    except Exception as e:
        print(f"❌ Hata: {e}")
    finally:
        # TEMİZLİK: Her şeyi eski haline getir (Çok önemli!)
        GenerationMixin.generate = original_generate
        print("🔧 generate fonksiyonu orijinal haline döndü.")
        gc.collect()
        torch.cuda.empty_cache()


# --- V3 İÇİN ÇALIŞTIR (KAŞİF MODU) ---
run_teacher_versioned(
    teacher_name="Cosmos_T1_v3",
    model_id="ytu-ce-cosmos/Turkish-Gemma-9b-T1",
    temp=0.8,
    top_p=0.95,
    max_new_tokens=256,
    do_sample=True
)

In [ ]:
# ==========================================
# HÜCRE 2: KESİN ÇÖZÜM (GenerationMixin Patch & Eksik Fonksiyon)
# ==========================================

from transformers.generation.utils import GenerationMixin
import gc, torch
import sys  # Gerekirse modül seviyesinde eklemek için

# 1. Eksik Fonksiyonu Tanımlıyoruz
def build_retry_prompt(question):
    return f"Soru: {question}\n\nÖnceki cevap formatı hatalıydı. Lütfen kurallara tam uyarak tekrar cevapla."

# 2. Fonksiyonu Global Alana Ekliyoruz (Bu kritik!)
# Böylece Hücre 1'deki kod bu ismi aradığında bulabilecek.
globals()['build_retry_prompt'] = build_retry_prompt

def run_teacher_versioned(teacher_name, model_id, temp, top_p, max_new_tokens, do_sample=True):
    """
    Hücre 1'e dokunmadan, GenerationMixin üzerindeki generate fonksiyonunu
    geçici olarak değiştirir. Bu yöntem en garantisidir.
    """

    # Asıl generate fonksiyonunun gerçek sahibi GenerationMixin'dir.
    original_generate = GenerationMixin.generate

    # Kendi ayarlarımızı zorlayan fonksiyonu yazıyoruz
    def patched_generate(self, *args, **kwargs):
        # Parametreleri eziyoruz (Override)
        kwargs["do_sample"] = do_sample
        kwargs["temperature"] = temp
        kwargs["top_p"] = top_p
        kwargs["max_new_tokens"] = max_new_tokens

        # Orijinal fonksiyonu yeni ayarlarla çağırıyoruz
        return original_generate(self, *args, **kwargs)

    # YAMALAMA: Artık tüm modeller bu fonksiyonu kullanacak
    GenerationMixin.generate = patched_generate

    print(f"\n🔧 {teacher_name} AKTİF (GenerationMixin Patching):")
    print(f"   do_sample={do_sample} | temp={temp} | top_p={top_p} | max_new_tokens={max_new_tokens}")

    try:
        # Hücre 1'deki fonksiyonu çalıştır
        run_teacher_specific(teacher_name=teacher_name, model_id=model_id)
    except Exception as e:
        print(f"❌ Hata: {e}")
    finally:
        # TEMİZLİK: Her şeyi eski haline getir (Çok önemli!)
        GenerationMixin.generate = original_generate
        print("🔧 generate fonksiyonu orijinal haline döndü.")
        gc.collect()
        torch.cuda.empty_cache()


# --- V3 İÇİN ÇALIŞTIR (KAŞİF MODU) ---
run_teacher_versioned(
    teacher_name="Cosmos_T1_v3",
    model_id="ytu-ce-cosmos/Turkish-Gemma-9b-T1",
    temp=0.6,
    top_p=0.9,
    max_new_tokens=128,
    do_sample=True
)

In [ ]:
import os, torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

BASE_PATH = "/content/drive/MyDrive/Proje_Distill"
DATA_PATH = os.path.join(BASE_PATH, "distill_Cosmos_T1_v3.jsonl")
OUT_DIR   = os.path.join(BASE_PATH, "adapter_qwen_student")

STUDENT_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"

dataset = load_dataset("json", data_files=DATA_PATH, split="train")

tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"   # 🔥 Qwen için önemli

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    STUDENT_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","mlp.up_proj","mlp.down_proj","mlp.gate_proj"]
)

model = get_peft_model(model, lora_config)

MAX_LEN = 1024

def build_prompt(question):
    example_output = (
        "<think>\n"
        "Alan = Uzunluk x Genişlik\n"
        "Alan = 10 * 5 = 50\n"
        "</think>\n"
        "Final Answer: 50"
    )

    return (
        f"Soru: {question}\n\n"
        "GÖREV: Adım adım düşün ve soruyu çöz.\n"
        "KURALLAR:\n"
        "1. Önce <think> etiketleri arasında işlemleri yap.\n"
        "2. Bulduğun sonucu en sona 'Final Answer: <sayı>' formatında yaz.\n"
        "3. ⚠️ Örneği kopyalama.\n\n"
        "ÖRNEK FORMAT:\n"
        f"{example_output}\n\n"
    )

def preprocess(ex):
    q = ex["question"].strip()
    a = ex["answer"].strip()

    prompt = build_prompt(q)
    full_text = prompt + a + tokenizer.eos_token

    tokenized = tokenizer(full_text, truncation=True, max_length=MAX_LEN)
    prompt_ids = tokenizer(prompt, truncation=True, max_length=MAX_LEN)["input_ids"]

    labels = tokenized["input_ids"].copy()
    labels[:len(prompt_ids)] = [-100] * len(prompt_ids)

    tokenized["labels"] = labels
    return tokenized

tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    pad_to_multiple_of=8
)

training_args = TrainingArguments(
    output_dir=OUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    gradient_checkpointing=True,
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),
    optim="paged_adamw_8bit",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()

model.save_pretrained(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)
print("✅ Student LoRA kaydedildi")



In [ ]:
!pip install -U bitsandbytes transformers accelerate peft datasets

In [ ]:
import os, torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

BASE_PATH = "/content/drive/MyDrive/Proje_Distill"
DATA_PATH = os.path.join(BASE_PATH, "distill_Cosmos_T1_v3.jsonl")
OUT_DIR   = os.path.join(BASE_PATH, "adapter_qwen_student")

STUDENT_MODEL = "Metin/gemma-2b-tr-inst"

dataset = load_dataset("json", data_files=DATA_PATH, split="train")

tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"   # 🔥 Qwen için önemli

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    STUDENT_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","mlp.up_proj","mlp.down_proj","mlp.gate_proj"]
)

model = get_peft_model(model, lora_config)

MAX_LEN = 1024

def build_prompt(question):
    example_output = (
        "<think>\n"
        "Alan = Uzunluk x Genişlik\n"
        "Alan = 10 * 5 = 50\n"
        "</think>\n"
        "Final Answer: 50"
    )

    return (
        f"Soru: {question}\n\n"
        "GÖREV: Adım adım düşün ve soruyu çöz.\n"
        "KURALLAR:\n"
        "1. Önce <think> etiketleri arasında işlemleri yap.\n"
        "2. Bulduğun sonucu en sona 'Final Answer: <sayı>' formatında yaz.\n"
        "3. ⚠️ Örneği kopyalama.\n\n"
        "ÖRNEK FORMAT:\n"
        f"{example_output}\n\n"
    )

def preprocess(ex):
    q = ex["question"].strip()
    a = ex["answer"].strip()

    prompt = build_prompt(q)
    full_text = prompt + a + tokenizer.eos_token

    tokenized = tokenizer(full_text, truncation=True, max_length=MAX_LEN)
    prompt_ids = tokenizer(prompt, truncation=True, max_length=MAX_LEN)["input_ids"]

    labels = tokenized["input_ids"].copy()
    labels[:len(prompt_ids)] = [-100] * len(prompt_ids)

    tokenized["labels"] = labels
    return tokenized

tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    pad_to_multiple_of=8
)

training_args = TrainingArguments(
    output_dir=OUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    gradient_checkpointing=True,
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),
    optim="paged_adamw_8bit",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()

model.save_pretrained(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)
print("✅ Student LoRA kaydedildi")

In [ ]:
# ==========================================
# STUDENT TRAIN (LoRA SFT) - FINAL (Teacher-aligned + VRAM Friendly)
# Dynamic Padding + Correct Collator
# ==========================================
import os, torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# =========================
# PATHS
# =========================
BASE_PATH = "/content/drive/MyDrive/Proje_Distill"
TRAIN_PATH = os.path.join(BASE_PATH, "distill_Cosmos_Llama_Guncel.jsonl")

STUDENT_MODEL = "Metin/gemma-2b-tr-inst"
OUT_ADAPTER_DIR = os.path.join(BASE_PATH, "adapter_gemma2b_LLama_dynamicpad")

# =========================
# TRAIN PARAMS
# =========================
MAX_LEN = 1024
BATCH_SIZE = 2
GRAD_ACC = 8
LR = 2e-4
EPOCHS = 2
WARMUP_RATIO = 0.03
LOG_STEPS = 10
SAVE_STEPS = 200

# =========================
# TEACHER İLE AYNI PROMPT (DÜZ METİN)
# =========================
def build_prompt(question: str):
    example_output = (
        "<think>\n"
        "Alan = Uzunluk x Genişlik\n"
        "Alan = 10 * 5 = 50\n"
        "</think>\n"
        "Final Answer: 50"
    )

    return (
        f"Soru: {question}\n\n"
        "GÖREV: Adım adım düşün ve soruyu çöz.\n"
        "KURALLAR:\n"
        "1. Önce <think> etiketleri arasında işlemleri yap.\n"
        "2. Bulduğun sonucu en sona 'Final Answer: <sayı>' formatında yaz.\n"
        "3. ⚠️ DİKKAT: Aşağıdaki örnek sadece formatı göstermek içindir. Örnekteki sayıları ASLA kopyalama! Kendi bulduğun sonucu yaz!\n\n"
        "ÖRNEK FORMAT:\n"
        f"{example_output}\n\n"
    )

# =========================
# LOAD DATASET
# =========================
ds = load_dataset("json", data_files=TRAIN_PATH, split="train")
print("✅ Train size:", len(ds))

# =========================
# TOKENIZER
# =========================
tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL, use_fast=True)
tokenizer.padding_side = "right"   # train için doğru
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# =========================
# MODEL (4bit) & LoRA
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
)

base_model = AutoModelForCausalLM.from_pretrained(
    STUDENT_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

base_model.config.use_cache = False  # gradient checkpointing için şart
base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

model = get_peft_model(base_model, lora_config)

# ✅ EKLENDİ: padding id uyumu
model.config.pad_token_id = tokenizer.pad_token_id

model.print_trainable_parameters()

# =========================
# PREPROCESS (Dynamic padding için padding yok!)
# =========================
def preprocess(ex):
    q = str(ex["question"]).strip()
    a = str(ex["answer"]).strip()

    prompt = build_prompt(q)

    # Teacher output formatı zaten:
    # <think> ... </think>\nFinal Answer: ...
    full_text = prompt + a + tokenizer.eos_token

    tok_full = tokenizer(
        full_text,
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
        add_special_tokens=False
    )

    tok_prompt = tokenizer(
        prompt,
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
        add_special_tokens=False
    )

    input_ids = tok_full["input_ids"]
    attention_mask = tok_full["attention_mask"]

    labels = input_ids.copy()

    # Prompt kısmını loss'tan çıkar
    prompt_len = len(tok_prompt["input_ids"])
    mask_len = min(prompt_len, len(labels))
    labels[:mask_len] = [-100] * mask_len

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

ds_tok = ds.map(preprocess, remove_columns=ds.column_names)

# =========================
# COLLATOR (DOĞRU OLAN)
# Dynamic padding + label padding = -100
# =========================
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    pad_to_multiple_of=8,
    label_pad_token_id=-100,
    return_tensors="pt"
)

# =========================
# TRAIN ARGS
# =========================
training_args = TrainingArguments(
    output_dir=OUT_ADAPTER_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    warmup_ratio=WARMUP_RATIO,
    logging_steps=LOG_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=2,
    gradient_checkpointing=True,   # 🔥 VRAM azaltır
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    optim="paged_adamw_8bit",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tok,
    data_collator=data_collator
)

# =========================
# TRAIN
# =========================
trainer.train()

# =========================
# SAVE
# =========================
model.save_pretrained(OUT_ADAPTER_DIR)
tokenizer.save_pretrained(OUT_ADAPTER_DIR)

print("✅ Training bitti!")
print("📌 Adapter kaydedildi:", OUT_ADAPTER_DIR)

In [ ]:
import os, json, re, torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from tqdm import tqdm

# =========================
# PATHS
# =========================
BASE_PATH = "/content/drive/MyDrive/Proje_Distill"

TEST_PATH = os.path.join(BASE_PATH, "test_500_final.jsonl")
ADAPTER_DIR = os.path.join(BASE_PATH, "adapter_gemma2b_LLama_dynamicpad")

STUDENT_MODEL = "Metin/gemma-2b-tr-inst"
OUT_PRED_PATH = os.path.join(BASE_PATH, "eval_results_test500.jsonl")

MAX_NEW_TOKENS = 512
RETRY_TOKENS = 128

# =========================
# PROMPT (TRAIN İLE BİREBİR)
# =========================
def build_prompt(question: str):
    example_output = (
        "<think>\n"
        "Alan = Uzunluk x Genişlik\n"
        "Alan = 10 * 5 = 50\n"
        "</think>\n"
        "Final Answer: 50"
    )

    return (
        f"Soru: {question}\n\n"
        "GÖREV: Adım adım düşün ve soruyu çöz.\n"
        "KURALLAR:\n"
        "1. Önce <think> etiketleri arasında işlemleri yap.\n"
        "2. Bulduğun sonucu en sona 'Final Answer: <sayı>' formatında yaz.\n"
        "3. ⚠️ DİKKAT: Aşağıdaki örnek sadece format içindir. Örnekteki sayıları ASLA kopyalama.\n\n"
        "ÖRNEK FORMAT:\n"
        f"{example_output}\n\n"
    )

def build_retry_prompt(question: str):
    return (
        f"Soru: {question}\n\n"
        "SADECE şu formatta cevap ver:\n"
        "<think>\n...\n</think>\n"
        "Final Answer: <sayı>\n"
        "Başka hiçbir şey yazma."
    )

# =========================
# NUMBER EXTRACTION
# =========================
def safe_float(x):
    try:
        return float(str(x).strip().replace(",", "."))
    except:
        return None

def extract_final_number(text: str):
    m = re.search(r"(Final\s*Answer|Cevap|Sonuç)\s*:?\s*(.*)", text, flags=re.IGNORECASE)
    if m:
        tail = m.group(2)
        m2 = re.search(r"([-+]?\d+(?:[.,]\d+)?)", tail)
        if m2:
            return m2.group(1)
    return None

def fallback_last_number(text: str):
    nums = re.findall(r"([-+]?\d+(?:[.,]\d+)?)", text)
    return nums[-1] if nums else None

def get_final_number(text):
    n = extract_final_number(text)
    if n is None:
        n = fallback_last_number(text)
    return n

# =========================
# MODEL LOAD
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
)

print("⏳ Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL, use_fast=True)
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("⏳ Base model yükleniyor...")
base_model = AutoModelForCausalLM.from_pretrained(
    STUDENT_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

print("⏳ Adapter yükleniyor...")
model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
model.eval()
model.config.pad_token_id = tokenizer.pad_token_id

print("✅ Model hazır")

# =========================
# LOAD TEST DATA
# =========================
test_ds = load_dataset("json", data_files=TEST_PATH, split="train")
print("✅ Test size:", len(test_ds))

# =========================
# EVAL LOOP
# =========================
correct = 0
total = 0
retried = 0
invalid_gt = 0
invalid_pred = 0

with open(OUT_PRED_PATH, "w", encoding="utf-8") as fout:
    for i, ex in enumerate(tqdm(test_ds)):
        q = ex["question"]
        gt_text = ex["answer"]

        gt_num = safe_float(get_final_number(gt_text))
        if gt_num is None:
            invalid_gt += 1
            continue

        prompt = build_prompt(q)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        gen = out[0][inputs["input_ids"].shape[1]:]
        pred_text = tokenizer.decode(gen, skip_special_tokens=True).strip()

        pred_num = safe_float(get_final_number(pred_text))

        # Retry if needed
        did_retry = False
        if pred_num is None:
            did_retry = True
            retried += 1

            retry_prompt = build_retry_prompt(q)
            inp2 = tokenizer(retry_prompt, return_tensors="pt").to(model.device)

            with torch.no_grad():
                out2 = model.generate(
                    **inp2,
                    max_new_tokens=RETRY_TOKENS,
                    do_sample=False,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            gen2 = out2[0][inp2["input_ids"].shape[1]:]
            retry_text = tokenizer.decode(gen2, skip_special_tokens=True).strip()

            pred_text += "\n\n[RETRY]\n" + retry_text
            pred_num = safe_float(get_final_number(retry_text))

        if pred_num is None:
            invalid_pred += 1

        is_correct = pred_num is not None and abs(pred_num - gt_num) < 1e-4

        total += 1
        correct += int(is_correct)

        fout.write(json.dumps({
            "idx": i,
            "question": q,
            "gt_num": gt_num,
            "pred_num": pred_num,
            "correct": is_correct,
            "retried": did_retry,
            "pred_text": pred_text
        }, ensure_ascii=False) + "\n")

# =========================
# METRICS
# =========================
acc = correct / total * 100 if total else 0

print("\n📊 RESULTS")
print(f"Accuracy: %{acc:.2f}")
print(f"Correct: {correct}/{total}")
print(f"Retry used: {retried}")
print(f"Invalid GT: {invalid_gt}")
print(f"Invalid Pred: {invalid_pred}")
print(f"Saved to: {OUT_PRED_PATH}")

In [ ]:
!pip install -U torch torchvision torchaudio

In [ ]:
# ==========================================
# STUDENT TRAIN (LoRA SFT) - FINAL (Teacher-aligned + VRAM Friendly)
# Dynamic Padding + Correct Collator
# ==========================================
import os, torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# =========================
# PATHS
# =========================
BASE_PATH = "/content/drive/MyDrive/Proje_Distill"
TRAIN_PATH = os.path.join(BASE_PATH, "distill_Cosmos_T1_v3_guncel.jsonl")

STUDENT_MODEL = "Metin/gemma-2b-tr-inst"
OUT_ADAPTER_DIR = os.path.join(BASE_PATH, "adapter_gemma2b_CosmosT1v3_dynamicpad")

# =========================
# TRAIN PARAMS
# =========================
MAX_LEN = 1024
BATCH_SIZE = 2
GRAD_ACC = 8
LR = 2e-4
EPOCHS = 2
WARMUP_RATIO = 0.03
LOG_STEPS = 10
SAVE_STEPS = 200

# =========================
# TEACHER İLE AYNI PROMPT (DÜZ METİN)
# =========================
def build_prompt(question: str):
    example_output = (
        "<think>\n"
        "Alan = Uzunluk x Genişlik\n"
        "Alan = 10 * 5 = 50\n"
        "</think>\n"
        "Final Answer: 50"
    )

    return (
        f"Soru: {question}\n\n"
        "GÖREV: Adım adım düşün ve soruyu çöz.\n"
        "KURALLAR:\n"
        "1. Önce <think> etiketleri arasında işlemleri yap.\n"
        "2. Bulduğun sonucu en sona 'Final Answer: <sayı>' formatında yaz.\n"
        "3. ⚠️ DİKKAT: Aşağıdaki örnek sadece formatı göstermek içindir. Örnekteki sayıları ASLA kopyalama! Kendi bulduğun sonucu yaz!\n\n"
        "ÖRNEK FORMAT:\n"
        f"{example_output}\n\n"
    )

# =========================
# LOAD DATASET
# =========================
ds = load_dataset("json", data_files=TRAIN_PATH, split="train")
print("✅ Train size:", len(ds))

# =========================
# TOKENIZER
# =========================
tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL, use_fast=True)
tokenizer.padding_side = "right"   # train için doğru
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# =========================
# MODEL (4bit) & LoRA
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
)

base_model = AutoModelForCausalLM.from_pretrained(
    STUDENT_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

base_model.config.use_cache = False  # gradient checkpointing için şart
base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

model = get_peft_model(base_model, lora_config)

# ✅ EKLENDİ: padding id uyumu
model.config.pad_token_id = tokenizer.pad_token_id

model.print_trainable_parameters()

# =========================
# PREPROCESS (Dynamic padding için padding yok!)
# =========================
def preprocess(ex):
    q = str(ex["question"]).strip()
    a = str(ex["answer"]).strip()

    prompt = build_prompt(q)

    # Teacher output formatı zaten:
    # <think> ... </think>\nFinal Answer: ...
    full_text = prompt + a + tokenizer.eos_token

    tok_full = tokenizer(
        full_text,
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
        add_special_tokens=False
    )

    tok_prompt = tokenizer(
        prompt,
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
        add_special_tokens=False
    )

    input_ids = tok_full["input_ids"]
    attention_mask = tok_full["attention_mask"]

    labels = input_ids.copy()

    # Prompt kısmını loss'tan çıkar
    prompt_len = len(tok_prompt["input_ids"])
    mask_len = min(prompt_len, len(labels))
    labels[:mask_len] = [-100] * mask_len

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

ds_tok = ds.map(preprocess, remove_columns=ds.column_names)

# =========================
# COLLATOR (DOĞRU OLAN)
# Dynamic padding + label padding = -100
# =========================
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    pad_to_multiple_of=8,
    label_pad_token_id=-100,
    return_tensors="pt"
)

# =========================
# TRAIN ARGS
# =========================
training_args = TrainingArguments(
    output_dir=OUT_ADAPTER_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    warmup_ratio=WARMUP_RATIO,
    logging_steps=LOG_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=2,
    gradient_checkpointing=True,   # 🔥 VRAM azaltır
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    optim="paged_adamw_8bit",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tok,
    data_collator=data_collator
)

# =========================
# TRAIN
# =========================
trainer.train()

# =========================
# SAVE
# =========================
model.save_pretrained(OUT_ADAPTER_DIR)
tokenizer.save_pretrained(OUT_ADAPTER_DIR)

print("✅ Training bitti!")
print("📌 Adapter kaydedildi:", OUT_ADAPTER_DIR)

In [ ]:
import os, json, re, torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from tqdm import tqdm

# =========================
# PATHS
# =========================
BASE_PATH = "/content/drive/MyDrive/Proje_Distill"

TEST_PATH = os.path.join(BASE_PATH, "test_500_final.jsonl")
ADAPTER_DIR = os.path.join(BASE_PATH, "adapter_gemma2b_CosmosT1v3_dynamicpad")

STUDENT_MODEL = "Metin/gemma-2b-tr-inst"
OUT_PRED_PATH = os.path.join(BASE_PATH, "eval_results_test500.jsonl")

MAX_NEW_TOKENS = 512
RETRY_TOKENS = 128

# =========================
# PROMPT (TRAIN İLE BİREBİR)
# =========================
def build_prompt(question: str):
    example_output = (
        "<think>\n"
        "Alan = Uzunluk x Genişlik\n"
        "Alan = 10 * 5 = 50\n"
        "</think>\n"
        "Final Answer: 50"
    )

    return (
        f"Soru: {question}\n\n"
        "GÖREV: Adım adım düşün ve soruyu çöz.\n"
        "KURALLAR:\n"
        "1. Önce <think> etiketleri arasında işlemleri yap.\n"
        "2. Bulduğun sonucu en sona 'Final Answer: <sayı>' formatında yaz.\n"
        "3. ⚠️ DİKKAT: Aşağıdaki örnek sadece format içindir. Örnekteki sayıları ASLA kopyalama.\n\n"
        "ÖRNEK FORMAT:\n"
        f"{example_output}\n\n"
    )

def build_retry_prompt(question: str):
    return (
        f"Soru: {question}\n\n"
        "SADECE şu formatta cevap ver:\n"
        "<think>\n...\n</think>\n"
        "Final Answer: <sayı>\n"
        "Başka hiçbir şey yazma."
    )

# =========================
# NUMBER EXTRACTION
# =========================
def safe_float(x):
    try:
        return float(str(x).strip().replace(",", "."))
    except:
        return None

def extract_final_number(text: str):
    m = re.search(r"(Final\s*Answer|Cevap|Sonuç)\s*:?\s*(.*)", text, flags=re.IGNORECASE)
    if m:
        tail = m.group(2)
        m2 = re.search(r"([-+]?\d+(?:[.,]\d+)?)", tail)
        if m2:
            return m2.group(1)
    return None

def fallback_last_number(text: str):
    nums = re.findall(r"([-+]?\d+(?:[.,]\d+)?)", text)
    return nums[-1] if nums else None

def get_final_number(text):
    n = extract_final_number(text)
    if n is None:
        n = fallback_last_number(text)
    return n

# =========================
# MODEL LOAD
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
)

print("⏳ Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL, use_fast=True)
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("⏳ Base model yükleniyor...")
base_model = AutoModelForCausalLM.from_pretrained(
    STUDENT_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

print("⏳ Adapter yükleniyor...")
model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
model.eval()
model.config.pad_token_id = tokenizer.pad_token_id

print("✅ Model hazır")

# =========================
# LOAD TEST DATA
# =========================
test_ds = load_dataset("json", data_files=TEST_PATH, split="train")
print("✅ Test size:", len(test_ds))

# =========================
# EVAL LOOP
# =========================
correct = 0
total = 0
retried = 0
invalid_gt = 0
invalid_pred = 0

with open(OUT_PRED_PATH, "w", encoding="utf-8") as fout:
    for i, ex in enumerate(tqdm(test_ds)):
        q = ex["question"]
        gt_text = ex["answer"]

        gt_num = safe_float(get_final_number(gt_text))
        if gt_num is None:
            invalid_gt += 1
            continue

        prompt = build_prompt(q)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        gen = out[0][inputs["input_ids"].shape[1]:]
        pred_text = tokenizer.decode(gen, skip_special_tokens=True).strip()

        pred_num = safe_float(get_final_number(pred_text))

        # Retry if needed
        did_retry = False
        if pred_num is None:
            did_retry = True
            retried += 1

            retry_prompt = build_retry_prompt(q)
            inp2 = tokenizer(retry_prompt, return_tensors="pt").to(model.device)

            with torch.no_grad():
                out2 = model.generate(
                    **inp2,
                    max_new_tokens=RETRY_TOKENS,
                    do_sample=False,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            gen2 = out2[0][inp2["input_ids"].shape[1]:]
            retry_text = tokenizer.decode(gen2, skip_special_tokens=True).strip()

            pred_text += "\n\n[RETRY]\n" + retry_text
            pred_num = safe_float(get_final_number(retry_text))

        if pred_num is None:
            invalid_pred += 1

        is_correct = pred_num is not None and abs(pred_num - gt_num) < 1e-4

        total += 1
        correct += int(is_correct)

        fout.write(json.dumps({
            "idx": i,
            "question": q,
            "gt_num": gt_num,
            "pred_num": pred_num,
            "correct": is_correct,
            "retried": did_retry,
            "pred_text": pred_text
        }, ensure_ascii=False) + "\n")

# =========================
# METRICS
# =========================
acc = correct / total * 100 if total else 0

print("\n📊 RESULTS")
print(f"Accuracy: %{acc:.2f}")
print(f"Correct: {correct}/{total}")
print(f"Retry used: {retried}")
print(f"Invalid GT: {invalid_gt}")
print(f"Invalid Pred: {invalid_pred}")
print(f"Saved to: {OUT_PRED_PATH}")

In [ ]:
import os, json, re, torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm

# =========================
# PATHS
# =========================
BASE_PATH = "/content/drive/MyDrive/Proje_Distill"

TEST_PATH = os.path.join(BASE_PATH, "test_500_final.jsonl")

# 👉 BURAYA TEACHER MODEL ADINI YAZ
TEACHER_MODEL = "ytu-ce-cosmos/Turkish-Llama-8b-Instruct-v0.1"

OUT_PRED_PATH = os.path.join(BASE_PATH, "eval_results_cosmost1teacher_test500.jsonl")

MAX_NEW_TOKENS = 512
RETRY_TOKENS = 128

# =========================
# PROMPT (TRAIN İLE BİREBİR)
# =========================
def build_prompt(question: str):
    example_output = (
        "<think>\n"
        "Alan = Uzunluk x Genişlik\n"
        "Alan = 10 * 5 = 50\n"
        "</think>\n"
        "Final Answer: 50"
    )

    return (
        f"Soru: {question}\n\n"
        "GÖREV: Adım adım düşün ve soruyu çöz.\n"
        "KURALLAR:\n"
        "1. Önce <think> etiketleri arasında işlemleri yap.\n"
        "2. Bulduğun sonucu en sona 'Final Answer: <sayı>' formatında yaz.\n"
        "3. ⚠️ DİKKAT: Aşağıdaki örnek sadece format içindir. Örnekteki sayıları ASLA kopyalama.\n\n"
        "ÖRNEK FORMAT:\n"
        f"{example_output}\n\n"
    )

def build_retry_prompt(question: str):
    return (
        f"Soru: {question}\n\n"
        "SADECE şu formatta cevap ver:\n"
        "<think>\n...\n</think>\n"
        "Final Answer: <sayı>\n"
        "Başka hiçbir şey yazma."
    )

# =========================
# NUMBER EXTRACTION
# =========================
def safe_float(x):
    try:
        return float(str(x).strip().replace(",", "."))
    except:
        return None

def extract_final_number(text: str):
    m = re.search(r"(Final\s*Answer|Cevap|Sonuç)\s*:?\s*(.*)", text, flags=re.IGNORECASE)
    if m:
        tail = m.group(2)
        m2 = re.search(r"([-+]?\d+(?:[.,]\d+)?)", tail)
        if m2:
            return m2.group(1)
    return None

def fallback_last_number(text: str):
    nums = re.findall(r"([-+]?\d+(?:[.,]\d+)?)", text)
    return nums[-1] if nums else None

def get_final_number(text):
    n = extract_final_number(text)
    if n is None:
        n = fallback_last_number(text)
    return n

# =========================
# MODEL LOAD (TEACHER)
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
)

print("⏳ Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(TEACHER_MODEL, use_fast=True)
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("⏳ Teacher model yükleniyor...")
model = AutoModelForCausalLM.from_pretrained(
    TEACHER_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

model.eval()
model.config.pad_token_id = tokenizer.pad_token_id

print("✅ Teacher model hazır")

# =========================
# LOAD TEST DATA
# =========================
test_ds = load_dataset("json", data_files=TEST_PATH, split="train")
print("✅ Test size:", len(test_ds))

# =========================
# EVAL LOOP
# =========================
correct = 0
total = 0
retried = 0
invalid_gt = 0
invalid_pred = 0

with open(OUT_PRED_PATH, "w", encoding="utf-8") as fout:
    for i, ex in enumerate(tqdm(test_ds)):
        q = ex["question"]
        gt_text = ex["answer"]

        gt_num = safe_float(get_final_number(gt_text))
        if gt_num is None:
            invalid_gt += 1
            continue

        prompt = build_prompt(q)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        gen = out[0][inputs["input_ids"].shape[1]:]
        pred_text = tokenizer.decode(gen, skip_special_tokens=True).strip()

        pred_num = safe_float(get_final_number(pred_text))

        # Retry if needed
        did_retry = False
        if pred_num is None:
            did_retry = True
            retried += 1

            retry_prompt = build_retry_prompt(q)
            inp2 = tokenizer(retry_prompt, return_tensors="pt").to(model.device)

            with torch.no_grad():
                out2 = model.generate(
                    **inp2,
                    max_new_tokens=RETRY_TOKENS,
                    do_sample=False,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            gen2 = out2[0][inp2["input_ids"].shape[1]:]
            retry_text = tokenizer.decode(gen2, skip_special_tokens=True).strip()

            pred_text += "\n\n[RETRY]\n" + retry_text
            pred_num = safe_float(get_final_number(retry_text))

        if pred_num is None:
            invalid_pred += 1

        is_correct = pred_num is not None and abs(pred_num - gt_num) < 1e-4

        total += 1
        correct += int(is_correct)

        fout.write(json.dumps({
            "idx": i,
            "question": q,
            "gt_num": gt_num,
            "pred_num": pred_num,
            "correct": is_correct,
            "retried": did_retry,
            "pred_text": pred_text
        }, ensure_ascii=False) + "\n")

# =========================
# METRICS
# =========================
acc = correct / total * 100 if total else 0

print("\n📊 RESULTS")
print(f"Accuracy: %{acc:.2f}")
print(f"Correct: {correct}/{total}")
print(f"Retry used: {retried}")
print(f"Invalid GT: {invalid_gt}")
print(f"Invalid Pred: {invalid_pred}")
print(f"Saved to: {OUT_PRED_PATH}")

In [ ]:
!pip install -U transformers huggingface_hub accelerate datasets bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
import os, json, re, torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm

# =========================
# PATHS
# =========================
BASE_PATH = "/content/drive/MyDrive/Proje_Distill"
TEST_PATH = os.path.join(BASE_PATH, "test_500_final.jsonl")

TEACHER_MODEL = "hosasmek/qwen2.5-7b-instruct-reasoning-tr"
OUT_PRED_PATH = os.path.join(BASE_PATH, "eval_results_qwenteacher_test500.jsonl")

MAX_NEW_TOKENS = 512
RETRY_TOKENS   = 64

# =========================
# NUMBER UTILS (STUDENT İLE AYNI)
# =========================
def safe_float(x):
    try:
        if x is None:
            return None
        return float(str(x).strip().replace(",", "."))
    except:
        return None

def extract_final_number(text):
    if not text:
        return None
    m = re.search(r"(?:Final\s*Answer|Cevap|Sonuç)\s*:?\s*(.*)", text, re.IGNORECASE)
    if m:
        tail = m.group(1)
        m2 = re.search(r"([-+]?\d+(?:[.,]\d+)?)", tail)
        if m2:
            return m2.group(1)
    return None

def fallback_last_number(text):
    nums = re.findall(r"([-+]?\d+(?:[.,]\d+)?)", str(text))
    return nums[-1] if nums else None

def get_final_number(text):
    n = extract_final_number(text)
    if n is None:
        n = fallback_last_number(text)
    return n

# =========================
# PROMPTS
# =========================
def build_prompt(question):
    example_output = (
        "<think>\n"
        "Alan = Uzunluk x Genişlik\n"
        "Alan = 10 * 5 = 50\n"
        "</think>\n"
        "Final Answer: 50"
    )
    return (
        f"Soru: {question}\n\n"
        "GÖREV: Adım adım düşün ve soruyu çöz.\n"
        "KURALLAR:\n"
        "1. Önce <think> etiketleri arasında işlemleri yap.\n"
        "2. Bulduğun sonucu en sona 'Final Answer: <sayı>' formatında yaz.\n"
        "3. Örnek sadece format içindir, sayıları kopyalama.\n\n"
        "ÖRNEK FORMAT:\n"
        f"{example_output}\n\n"
    )

def build_retry_prompt(question):
    return (
        f"Soru: {question}\n\n"
        "SADECE şu formatta cevap ver:\n"
        "<think>\n...\n</think>\n"
        "Final Answer: <sayı>"
    )

# =========================
# MODEL LOAD
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
)

print("⏳ Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(TEACHER_MODEL, use_fast=True)
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token

print("⏳ Teacher model yükleniyor...")
model = AutoModelForCausalLM.from_pretrained(
    TEACHER_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)
model.eval()
model.config.pad_token_id = tokenizer.pad_token_id

print("✅ Teacher model hazır")

# =========================
# LOAD TEST DATA
# =========================
test_ds = load_dataset("json", data_files=TEST_PATH, split="train")
print("✅ Test size:", len(test_ds))

# =========================
# 🔁 RESUME LOGIC (YENİ)
# =========================
start_index = 0
if os.path.exists(OUT_PRED_PATH):
    with open(OUT_PRED_PATH, "r", encoding="utf-8") as f:
        start_index = sum(1 for _ in f)

print(f"🔁 Resume from index: {start_index}")

# =========================
# EVAL LOOP
# =========================
correct = total = invalid_gt = invalid_pred = retried = 0

with open(OUT_PRED_PATH, "a", encoding="utf-8") as fout:
    for i in tqdm(range(start_index, len(test_ds))):
        ex = test_ds[i]
        q = ex["question"]
        gt_text = ex["answer"]

        gt_num = safe_float(get_final_number(gt_text))
        if gt_num is None:
            invalid_gt += 1
            continue

        prompt = build_prompt(q)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.inference_mode():
            out = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        gen = out[0][inputs["input_ids"].shape[1]:]
        pred_text = tokenizer.decode(gen, skip_special_tokens=True).strip()
        pred_num = safe_float(get_final_number(pred_text))

        did_retry = False
        if pred_num is None:
            did_retry = True
            retried += 1

            retry_prompt = build_retry_prompt(q)
            inp2 = tokenizer(retry_prompt, return_tensors="pt").to(model.device)

            with torch.inference_mode():
                out2 = model.generate(
                    **inp2,
                    max_new_tokens=RETRY_TOKENS,
                    do_sample=False,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            gen2 = out2[0][inp2["input_ids"].shape[1]:]
            retry_text = tokenizer.decode(gen2, skip_special_tokens=True).strip()
            pred_num = safe_float(get_final_number(retry_text))

        if pred_num is None:
            invalid_pred += 1

        is_correct = pred_num is not None and abs(pred_num - gt_num) < 1e-4
        total += 1
        correct += int(is_correct)

        fout.write(json.dumps({
            "idx": i,
            "question": q,
            "gt_num": gt_num,
            "pred_num": pred_num,
            "correct": is_correct,
            "retried": did_retry
        }, ensure_ascii=False) + "\n")
        fout.flush()

# =========================
# METRICS
# =========================
acc = correct / total * 100 if total else 0
print(f"\n✅ Accuracy: %{acc:.2f} ({correct}/{total})")
print(f"GT skip: {invalid_gt} | Pred invalid: {invalid_pred} | Retry: {retried}")
print(f"📁 Saved to: {OUT_PRED_PATH}")

⏳ Tokenizer yükleniyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Teacher model hazır


Generating train split: 0 examples [00:00, ? examples/s]

✅ Test size: 500
🔁 Resume from index: 0


 54%|█████▍    | 272/500 [3:14:14<2:42:00, 42.63s/it]